In [9]:
# Download all packages 
import pickle
import detritalpy.detritalFuncs as dFunc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import randrange
import random
import time
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,accuracy_score, precision_score, recall_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

In [10]:
data_que = {}

data_que['0.15-0.25'] = [['0.15-0.25 - 0 Sources.p','0.15-0.25 - 1 Sources.p','0.15-0.25 - 2 Sources.p','0.15-0.25 - 3 Sources.p',
                          '0.15-0.25 - 4 Sources.p','0.15-0.25 - 5 Sources.p','0.15-0.25 - 6 Sources.p','0.15-0.25 - 7 Sources.p'],
                         'New 0.15-0.25 Sources.p']

data_que['0.25-0.35'] = [['0.25-0.35 - 0 Sources.p','0.25-0.35 - 1 Sources.p','0.25-0.35 - 2 Sources.p','0.25-0.35 - 3 Sources.p',
                          '0.25-0.35 - 4 Sources.p','0.25-0.35 - 5 Sources.p','0.25-0.35 - 6 Sources.p','0.25-0.35 - 7 Sources.p'],
                         'New 0.25-0.35 Sources.p']

data_que['0.35-0.45'] = [['0.35-0.45 - 0 Sources.p','0.35-0.45 - 1 Sources.p','0.35-0.45 - 2 Sources.p','0.35-0.45 - 3 Sources.p',
                          '0.35-0.45 - 4 Sources.p','0.35-0.45 - 5 Sources.p','0.35-0.45 - 6 Sources.p','0.35-0.45 - 7 Sources.p'],
                         'New 0.35-0.45 Sources.p']

data_que['0.45-0.55'] = [['0.45-0.55 - 0 Sources.p','0.45-0.55 - 1 Sources.p','0.45-0.55 - 2 Sources.p','0.45-0.55 - 3 Sources.p',
                          '0.45-0.55 - 4 Sources.p','0.45-0.55 - 5 Sources.p','0.45-0.55 - 6 Sources.p','0.45-0.55 - 7 Sources.p'],
                         'New 0.45-0.55 Sources.p']

data_que['0.55-0.65'] = [['0.55-0.65 - 0 Sources.p','0.55-0.65 - 1 Sources.p','0.55-0.65 - 2 Sources.p','0.55-0.65 - 3 Sources.p',
                          '0.55-0.65 - 4 Sources.p','0.55-0.65 - 5 Sources.p','0.55-0.65 - 6 Sources.p','0.55-0.65 - 7 Sources.p'],
                         'New 0.55-0.65 Sources.p']

data_que['0.65-0.75'] = [['0.65-0.75 - 0 Sources.p','0.65-0.75 - 1 Sources.p','0.65-0.75 - 2 Sources.p','0.65-0.75 - 3 Sources.p',
                          '0.65-0.75 - 4 Sources.p','0.65-0.75 - 5 Sources.p','0.65-0.75 - 6 Sources.p','0.65-0.75 - 7 Sources.p'],
                         'New 0.65-0.75 Sources.p']

data_que['0.75-0.85'] = [['0.75-0.85 - 0 Sources.p','0.75-0.85 - 1 Sources.p','0.75-0.85 - 2 Sources.p','0.75-0.85 - 3 Sources.p',
                          '0.75-0.85 - 4 Sources.p','0.75-0.85 - 5 Sources.p','0.75-0.85 - 6 Sources.p','0.75-0.85 - 7 Sources.p'],
                         'New 0.75-0.85 Sources.p']

data_que['0.85-0.95'] = [['0.85-0.95 - 0 Sources.p','0.85-0.95 - 1 Sources.p','0.85-0.95 - 2 Sources.p','0.85-0.95 - 3 Sources.p',
                          '0.85-0.95 - 4 Sources.p','0.85-0.95 - 5 Sources.p','0.85-0.95 - 6 Sources.p','0.85-0.95 - 7 Sources.p'],
                         'New 0.85-0.95 Sources.p']

                          
data_que['0.95-0.99'] = [['0.95-0.99 - 0 Sources.p','0.95-0.99 - 1 Sources.p','0.95-0.99 - 2 Sources.p','0.95-0.99 - 3 Sources.p',
                          '0.95-0.99 - 4 Sources.p','0.95-0.99 - 5 Sources.p','0.95-0.99 - 6 Sources.p','0.95-0.99 - 7 Sources.p'],
                         'New 0.95-0.99 Sources.p']

In [ ]:
# USER INPUTS
################################################################################################

# Number of times the ML pipeline will be trained and tested on the selected data
num_repeats = 10
# Place the TPOT pipeline from your code or example below:
TPOT_pipeline = MultinomialNB(alpha=0.001, fit_prior=False)

################################################################################################
################################################################################################
# Set up the loop
hit_vals = np.round(np.logspace(1,3,num=35),0)
# Load in data for each similarity interval
for data_pair in data_que:
    source_data = data_que[data_pair][0]
    init_data = {}
    for source in source_data:
        load_data = pickle.load(open(source,'rb'))
        init_data.update(load_data)
    OG_sources = pickle.load(open(data_que[data_pair][1],'rb'))

    # Initial final storage for data pair
    results_total = {}
    R2_totals = {}
    RF_totals = {}
    #######################################################################################
    # Setting up loops and selecting keys
    # w is for grains
    for w in hit_vals:
        round_start = time.time()
        # t is for sources
        for t in range(2,9):
            # AutoML storage
            ###################
            f1_bank = []
            acc_bank = []
            prec_bank = []
            recall_bank = []
            sim_bank = []
            # R2 Storage
            ###################
            R2_F1_bank = []
            R2_acc_bank = []
            R2_prec_bank = []
            R2_recall_bank = []
            # Random Forest Storage
            ####################
            RF_F1_bank = []
            RF_acc_bank = []
            RF_prec_bank = []
            RF_recall_bank = []
            ####################
            # Start loops
            start = time.time()
            for v in range(0,num_repeats):
                # temp spot to hold the data
                data_hold = {}
                # Randomly select keys to compare
                select_keys = random.sample(range(0,8),t)
                for key in init_data:
                    if key[1] == w and key[0] in select_keys:
                        data_hold[key] = init_data[key]
                # Now we need to go through and separate the data and assign labels
        ########################################################################################
                # Data structuring
                ID_list = []
                KDE_list = []
                for entry in data_hold:
                    source_ID = entry[0]
                    for KDE in data_hold[entry]:
                        KDE_list.append(KDE)
                        ID_list.append(source_ID)
                final_df = pd.DataFrame()
                final_df['ID'] = ID_list
                final_df['KDE'] = KDE_list
        ########################################################################################

        #########################################################################################
                # All the ML stuff ###############################
                train_set,test_set = train_test_split(final_df,test_size = 0.2, random_state = randrange(42))
                train_KDE = list(train_set['KDE'])
                train_labels = list(train_set['ID'])
                test_KDE = list(test_set['KDE'])
                test_labels = list(test_set['ID'])
                enc = OrdinalEncoder()
                train_labels = np.array(train_labels)
                train_labels = train_labels.reshape(-1,1)
                train_labels = enc.fit_transform(train_labels)
                test_labels = np.array(test_labels)
                test_labels = test_labels.reshape(-1,1)
                test_labels = enc.fit_transform(test_labels)
                exported_pipeline = TPOT_pipeline
                # Fix random state in exported estimator
                if hasattr(exported_pipeline, 'random_state'):
                    setattr(exported_pipeline, 'random_state', 42)
                model = exported_pipeline.fit(train_KDE,train_labels.ravel())
                results = exported_pipeline.predict(test_KDE)

        ######### GET THE SCORES #################################################################
                F1 = f1_score(test_labels,results,average='weighted')
                acc = accuracy_score(test_labels,results)
                prec = precision_score(test_labels,results,average='weighted',zero_division = 0)
                rec = recall_score(test_labels,results,average='weighted',zero_division = 0)
                #final_sim_score = (sum(total_sim_scores)/len(total_sim_scores))
                f1_bank.append(F1)
                acc_bank.append(acc)
                prec_bank.append(prec)
                recall_bank.append(rec)
                #sim_bank.append(final_sim_score)

        ####### Random Forest Tests ############################################################
                RF_start = time.time()
                rf_clf = RandomForestClassifier()
                rf_model = rf_clf.fit(train_KDE,train_labels.ravel())
                rf_results = rf_clf.predict(test_KDE)

                # Get scores
                rf_f1 = f1_score(test_labels,rf_results,average='weighted')
                rf_acc = accuracy_score(test_labels,rf_results)
                rf_prec = precision_score(test_labels,rf_results,average='weighted',zero_division = 0)
                rf_rec = recall_score(test_labels,rf_results,average='weighted',zero_division = 0)
                #final_sim_score = (sum(total_sim_scores)/len(total_sim_scores))
                RF_F1_bank.append(rf_f1)
                RF_acc_bank.append(rf_acc)
                RF_prec_bank.append(rf_prec)
                RF_recall_bank.append(rf_rec)
                #sim_bank.append(final_sim_score)

        ######## GET R2 Scores ###################################################################
                R2_start = time.time()
                R2_preds = []
                R2_labels = []
                used_sources = {}
                for key in select_keys:
                    used_sources[key] = OG_sources[key]
                for index in test_set.index:
                    temp_dict = {}
                    test_KDE = test_set.loc[index]['KDE']
                    test_ID = test_set.loc[index]['ID']
                    R2_labels.append(test_ID)
                    for source in used_sources:
                        OG_source = OG_sources[source][0]
                        R2_score = dFunc.calcR2(test_KDE,OG_source)
                        temp_dict[source] = R2_score
                    max_val = max(temp_dict, key = temp_dict.get)
                    R2_preds.append(max_val)
                # Now we get the scores
                R2_F1 = f1_score(R2_labels,R2_preds,average='weighted')
                R2_acc = accuracy_score(R2_labels,R2_preds)
                R2_prec = precision_score(R2_labels,R2_preds,average='weighted',zero_division = 0)
                R2_rec = recall_score(R2_labels,R2_preds,average='weighted',zero_division = 0)
                R2_F1_bank.append(R2_F1)
                R2_acc_bank.append(R2_acc)
                R2_prec_bank.append(R2_prec)
                R2_recall_bank.append(R2_rec)
                
        ##########################################################################################
            end = time.time()
            total_elapsed_time = end-start   
            f1_final = sum(f1_bank)/len(f1_bank)
            print()
            print('AutoML - F1 Score:',round(f1_final,2))
            #sim_final = sum(sim_bank)/len(sim_bank)
            sim_final = None
            acc_final = sum(acc_bank)/len(acc_bank)
            prec_final =  sum(prec_bank)/len(prec_bank)
            recall_final = sum(recall_bank)/len(recall_bank)
            results_total[(t),w] = [f1_final,sim_final,acc_final,prec_final,recall_final,total_elapsed_time,f1_bank]
            #print('Finished ML',w,t)
    ##########################################################################################
            R2_end = time.time()
            final_sim_score = None
            R2_time = R2_end - R2_start
            R2_F1_final = sum(R2_F1_bank)/len(R2_F1_bank)
            print('R2 - F1 Score:',round(R2_F1_final,2))
            R2_acc_final = sum(R2_acc_bank)/len(R2_acc_bank)
            R2_prec_final = sum(R2_prec_bank)/len(R2_prec_bank)
            R2_recall_final = sum(R2_recall_bank)/len(R2_recall_bank)
            R2_totals[(t),w] = [R2_F1_final,final_sim_score,R2_acc_final,R2_prec_final,R2_recall_final,R2_time]
    ############################################################################################
            RF_end = time.time()
            final_sim_score = None
            RF_time = RF_end-RF_start
            RF_F1_final = sum(RF_F1_bank)/len(RF_F1_bank)
            print('Random Forest - F1 Score:',round(RF_F1_final,2))
            RF_acc_final = sum(RF_acc_bank)/len(RF_acc_bank)
            RF_prec_final = sum(RF_prec_bank)/len(RF_prec_bank)
            RF_recall_final = sum(RF_recall_bank)/len(RF_recall_bank)
            RF_totals[(t),w] = [RF_F1_final,final_sim_score,RF_acc_final,RF_prec_final,RF_recall_final,RF_time]

        round_end = time.time()
        print()
        print('Round Training Time:',round((round_end-round_start),2),' seconds.')
        print('Finished with Analyses:',w)
    ML_naming_id = 'Fixed '+data_pair+' AutoML Results.p'
    R2_naming_id = 'Fixed '+data_pair+' R2 Results.p'
    RF_naming_id = 'Fixed '+data_pair+' RF Results.p'
    pickle.dump(results_total,open(ML_naming_id,'wb'))
    pickle.dump(R2_totals,open(R2_naming_id,'wb'))
    pickle.dump(RF_totals,open(RF_naming_id,'wb'))
    print('Succesful save.')
    print('Deleting old data.')
    print()
    print('----------------------')
    del init_data